In [1]:
import pandas as pd
import numpy as np

In [2]:
import tensorflow as tf
tf.__version__

'2.18.0'

In [3]:
import nltk
import re
from nltk.corpus import stopwords

In [4]:
nltk.download('stopwords')
nltk.download('all')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | D

True

In [6]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 12.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1


In [8]:
!pip uninstall -y gensim numpy
!pip install --no-cache-dir gensim numpy

Found existing installation: gensim 4.3.3
Uninstalling gensim-4.3.3:
  Successfully uninstalled gensim-4.3.3
Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 172.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 164.9 MB/s eta 0:00:00


In [9]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [10]:
from tensorflow.keras.layers import Embedding  #embedding layer helps us with the word to word implementation
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

# Importing and preprocessing the data

In [11]:
df = pd.read_csv('Sentiment_Stock_data.csv')
df = df.head(5000)
df.head()

Unnamed: 0  Sentiment                                           Sentence
0           0          0  According to Gran , the company has no plans t...
1           1          1  For the last quarter of 2010 , Componenta 's n...
2           2          1  In the third quarter of 2010 , net sales incre...
3           3          1  Operating profit rose to EUR 13.1 mn from EUR ...
4           4          1  Operating profit totalled EUR 21.1 mn , up fro...

In [12]:
df = df[['Sentiment', 'Sentence']]

In [13]:
df.shape

(5000, 2)

In [14]:
df.isnull().sum()

Sentiment    0
Sentence     0
dtype: int64

In [15]:
# Dropping null values

df.dropna(inplace=True)

In [16]:
df.reset_index(inplace=True)

In [17]:
# specifying the independent and dependent features
X = df['Sentence']
y = df['Sentiment']

In [18]:
# copying X for preprocessing

sentences = X.copy()

In [19]:
nltk.download('wordnet')
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

unzip:  cannot find or open /usr/share/nltk_data/corpora/wordnet.zip, /usr/share/nltk_data/corpora/wordnet.zip.zip or /usr/share/nltk_data/corpora/wordnet.zip.ZIP.


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [20]:
# Lemmatization to convert words in sentences to their meaningful root

from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

corpus = []
for i in range(0, len(sentences)):
    text = re.sub('[^a-zA-Z]', ' ', sentences[i])
    text = text.lower()
    text = text.split()
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not')
    all_stopwords.remove('no')

    text = [lemmatizer.lemmatize(word) for word in text if not word in set(all_stopwords)]
    text = ' '.join(text)
    corpus.append(text)

In [21]:
corpus[0]

'according gran company no plan move production russia although company growing'

In [22]:
len(corpus)

5000

In [23]:
corpus

['according gran company no plan move production russia although company growing',
 'last quarter componenta net sale doubled eur eur period year earlier moved zero pre tax profit pre tax loss eur',
 'third quarter net sale increased eur mn operating profit eur mn',
 'operating profit rose eur mn eur mn corresponding period representing net sale',
 'operating profit totalled eur mn eur mn representing net sale',
 'finnish talentum report operating profit increased eur mn eur mn net sale totaled eur mn eur mn',
 'clothing retail chain sepp l sale increased eur mn operating profit rose eur mn eur mn',
 'consolidated net sale increased reach eur operating profit amounted eur compared loss eur prior year period',
 'foundry division report sale increased eur mn eur mn corresponding period sale machine shop division increased eur mn eur mn corresponding period',
 'helsinki afx share closed higher led nokia announced plan team sanyo manufacture g handset nokian tyre fourth quarter earnings re

In [24]:
X[0]

'According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing '

In [25]:
### Vocabulary size
voc_size=50000

In [26]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[19210, 4724, 2109, 41600, 33759, 1223, 38467, 44205, 47687, 2109, 16561],
 [37494,
  18401,
  17534,
  37383,
  29505,
  13533,
  6232,
  6232,
  48684,
  22901,
  19224,
  15243,
  514,
  37127,
  38291,
  41745,
  37127,
  38291,
  36578,
  6232],
 [27813, 18401, 37383, 29505, 43430, 6232, 43828, 23100, 41745, 6232, 43828],
 [23100,
  41745,
  41042,
  6232,
  43828,
  6232,
  43828,
  47588,
  48684,
  39260,
  37383,
  29505],
 [23100, 41745, 30263, 6232, 43828, 6232, 43828, 39260, 37383, 29505],
 [32967,
  7851,
  36265,
  23100,
  41745,
  43430,
  6232,
  43828,
  6232,
  43828,
  37383,
  29505,
  27063,
  6232,
  43828,
  6232,
  43828],
 [21536,
  5980,
  47864,
  40792,
  3018,
  29505,
  43430,
  6232,
  43828,
  23100,
  41745,
  41042,
  6232,
  43828,
  6232,
  43828],
 [49942,
  37383,
  29505,
  43430,
  27369,
  6232,
  23100,
  41745,
  20220,
  6232,
  33216,
  36578,
  6232,
  47871,
  22901,
  48684],
 [36971,
  7846,
  36265,
  29505,
  43430,
  6232,
  43828,


In [27]:
corpus[1]

'last quarter componenta net sale doubled eur eur period year earlier moved zero pre tax profit pre tax loss eur'

In [28]:
onehot_repr[1]

[37494,
 18401,
 17534,
 37383,
 29505,
 13533,
 6232,
 6232,
 48684,
 22901,
 19224,
 15243,
 514,
 37127,
 38291,
 41745,
 37127,
 38291,
 36578,
 6232]

### Word Embedding

In [29]:
#making the sentences equal length/size
sent_length=30
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[19210  4724  2109 ...     0     0     0]
 [37494 18401 17534 ...     0     0     0]
 [27813 18401 37383 ...     0     0     0]
 ...
 [25957 33445 37259 ...     0     0     0]
 [48825 15111 17017 ...     0     0     0]
 [18941 22070 36297 ...     0     0     0]]


In [30]:
embedded_docs[1]

array([37494, 18401, 17534, 37383, 29505, 13533,  6232,  6232, 48684,
       22901, 19224, 15243,   514, 37127, 38291, 41745, 37127, 38291,
       36578,  6232,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0], dtype=int32)

# Building the LSTM model

In [31]:
## Creating model
embedding_vector_features=40 ##features representation. Every index in embedded_docs will be represented by 40 dimensions
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(500))  #try different values
model.add(Dense(1,activation='sigmoid')) #sigmoid since the output is binary
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [32]:
len(embedded_docs),y.shape

(5000, (5000,))

# Model training

In [33]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [34]:
X_final.shape,y_final.shape

((5000, 30), (5000,))

In [35]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.3, random_state=42)

In [36]:



y_train

array([0, 0, 0, ..., 0, 0, 0])

In [37]:
# Model Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=75)

Epoch 1/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 31s 576ms/step - accuracy: 0.7072 - loss: 0.6226 - val_accuracy: 0.6920 - val_loss: 0.6085
Epoch 2/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 39s 528ms/step - accuracy: 0.7172 - loss: 0.5505 - val_accuracy: 0.8020 - val_loss: 0.4282
Epoch 3/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 43s 581ms/step - accuracy: 0.9087 - loss: 0.2377 - val_accuracy: 0.8273 - val_loss: 0.4475
Epoch 4/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 25s 535ms/step - accuracy: 0.9530 - loss: 0.1292 - val_accuracy: 0.8480 - val_loss: 0.4772
Epoch 5/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 25s 533ms/step - accuracy: 0.9708 - loss: 0.0928 - val_accuracy: 0.8520 - val_loss: 0.5438
Epoch 6/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 44s 587ms/step - accuracy: 0.9728 - loss: 0.0695 - val_accuracy: 0.8407 - val_loss: 0.7297
Epoch 7/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 41s 585ms/step - accuracy: 0.9778 - loss: 0.0757 - val_accuracy: 0.8533 - val_loss: 0.6274
Epoch 8/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 41s 580ms/step - accuracy: 0.9864 - loss: 0.0421 - val_accu

# Predictions and model performance

In [38]:
# Predicting the test data
y_pred = model.predict(X_test)
y_pred

47/47 ━━━━━━━━━━━━━━━━━━━━ 4s 71ms/step


array([[1.5868647e-03],
       [9.9830890e-01],
       [9.2212489e-04],
       ...,
       [4.8985329e-02],
       [9.9828011e-01],
       [2.2121798e-03]], dtype=float32)

In [39]:
# making the predictions binary

y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve
y_pred

array([[0],
       [1],
       [0],
       ...,
       [0],
       [1],
       [0]])

In [40]:
# Confusion matrix

from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[929, 109],
       [118, 344]])

In [41]:
# Getting the accuracy score

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8486666666666667

In [42]:
# Getting the classification report

from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.89      0.89      0.89      1038
           1       0.76      0.74      0.75       462

    accuracy                           0.85      1500
   macro avg       0.82      0.82      0.82      1500
weighted avg       0.85      0.85      0.85      1500



In [44]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.6 MB/s eta 0:00:00


In [45]:
import gradio as gr
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot

# Ensure model is trained before running this cell

# Define the same parameters as used in training
voc_size = 50000
sent_length = 30

# Preprocessing function for input text
def preprocess_text(text):
    onehot_repr = [one_hot(text, voc_size)]  # Convert text to one-hot encoding
    embedded_doc = pad_sequences(onehot_repr, padding='post', maxlen=sent_length)  # Pad sequence
    return np.array(embedded_doc)

# Prediction function for Gradio
def predict_sentiment(text):
    processed_text = preprocess_text(text)
    prediction = model.predict(processed_text)[0][0]  # Get prediction value

    # Convert probability into sentiment label
    sentiment = "Positive" if prediction > 0.5 else "Negative"
    return f"{sentiment} Sentiment ({prediction:.2f})"

# Create Gradio Interface
iface = gr.Interface(
    fn=predict_sentiment,
    inputs=gr.Textbox(lines=2, placeholder="Enter a stock-related sentence..."),
    outputs="text",
    title="Stock Sentiment Analysis",
    description="Enter a sentence related to stocks and get its sentiment (Positive/Negative)."
)

# Launch Gradio in Colab
iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4fa218a8d2f7803799.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


KeyboardInterrupt: 